In [1]:
import pandas as pd
import keras
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, BatchNormalization,Input
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import numpy as np
from numpy import exp
# Library for Gaussian process
# import GPy
##Library for visualization
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
import matplotlib;matplotlib.rcParams['figure.figsize'] = (8,6)
from sklearn.model_selection import train_test_split
import pylab 
import time

In [2]:
df_loc = pd.read_csv("../synthetic_data_simulations/2D_matern_nonstatiopnary_noise.csv", sep = ",")
N = len(df_loc)

s = np.vstack((df_loc["x"],df_loc["y"])).T

# ## Nonstationary data generation
# var1 = 10*np.sin(21*(((s[:,0] + s[:,1])**2)/2 -0.9)**4) * np.cos(17*((s[:,0] + 
#                                                                   s[:,1])/2 -0.9)) + ((
#                                                                 s[:,0] + s[:,1])/2 -0.9)/2 
# var2 = 10*np.sin(-31*(((s[:,0] + s[:,1])**2)/1.7 -0.45)**2) * np.cos(8.894*((s[:,0] + 
#                                                                   s[:,1])/2 -0.8)) + ((
#                                                                 s[:,0] + s[:,1])/2 -0.8)/3 
## Nonstationary data generation
s_bar = (s[:,0] + s[:,1])/2
var1 = np.sin(5*((s_bar) -0.9))*np.cos(25*((s_bar) -0.9)**4) + ((s_bar) -0.9)/2
var2 = np.sin(2*((s_bar) -0.9)) * np.cos(30*((s_bar) -0.9)**4) - ((s_bar) -0.9)/2 


## Nonstationary data generation
# var1 = np.sin((((df_loc["var1"])**2)/2 )) * np.cos(1.7*((df_loc["var1"])/2 -0.9)) + ((
#                                                                 s[:,0] + s[:,1])/2 -0.9)/2 
# var2 = np.sin(-(((df_loc["var2"] )**2)/1.7 )**2) * np.cos(1.894*((df_loc["var2"] )/2 -0.8)) + ((
#                                                                 s[:,0] + s[:,1])/2 -0.8)/3 

z1 = var1 + df_loc["var1"]
z2 = var2 + df_loc["var2"]

y = np.vstack((z1,z2)).T

In [3]:
num_basis = [2**2,5**2,9**2]
# num_basis = [2**2,5**2,9**2]
knots_1d = [np.linspace(0,1,int(np.sqrt(i))) for i in num_basis]
#knots_1d = [np.linspace(0,1,i) for i in num_basis]
##Wendland kernel
K = 0
phi = np.zeros((N, sum(num_basis)))
for res in range(len(num_basis)):
    theta = 1/np.sqrt(num_basis[res])*2.5
    knots_s1, knots_s2 = np.meshgrid(knots_1d[res],knots_1d[res])
    knots = np.column_stack((knots_s1.flatten(),knots_s2.flatten()))
    for i in range(num_basis[res]):
        d = np.linalg.norm(s-knots[i,:],axis=1)/theta
        for j in range(len(d)):
            if d[j] >= 0 and d[j] <= 1:
                phi[j,i + K] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
            else:
                phi[j,i + K] = 0
    K = K + num_basis[res]

### Train Test split
from sklearn.model_selection import train_test_split
indices = np.arange(N)
## Split the training and testing sets

In [4]:
phi.shape

(1200, 110)

## Cross validation 

In [5]:
## Functions for calculation of MSE and MAE
def mse(y_pred,y_true):
    mse = np.mean((y_pred-y_true)**2)
    return mse

def mae(y_pred,y_true):
    mae = np.mean(np.absolute(y_pred-y_true))
    return mae

mse_var1 = []
mse_var2 = []

In [7]:
mse_var1 = []
mse_var2 = []
start_time = time.time()
for i in range(10):

    s_train, s_test, encoder_train, encoder_test    , y_train, y_test= train_test_split(s, phi, y, 
                                                                                        test_size=0.33)
    # DeepKriging model for continuous data
    model = Sequential()
    # model.add(Dense(100, input_dim = 2,  kernel_initializer='he_uniform', activation='relu'))
    model.add(Dense(100, input_dim = encoder_train.shape[1],  kernel_initializer='he_uniform', activation='relu'))
    # model.add(Dropout(rate=0.5))
    # model.add(BatchNormalization())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(100, activation='relu'))
    #     model.add(Dense(100, activation='relu'))
    model.add(Dense(50, activation='relu'))
    # model.add(Dense(100, activation='relu'))
    #model.add(Dropout(rate=0.5))
    model.add(Dense(50, activation='relu'))
    #model.add(BatchNormalization())
    model.add(Dense(2, activation='linear'))
    NB_START_EPOCHS = 50 
    # NB_START_EPOCHS = 200  # Number of epochs we usually start to train with
    BATCH_SIZE = 64  
    fold_no = 1
    optimizer = keras.optimizers.Adam(lr=0.001)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mse','mae'])
    
    result = model.fit(encoder_train, y_train, 
                       validation_data=(encoder_test,y_test), epochs = 300, batch_size = 64, verbose = 2)

    callbacks = [EarlyStopping(monitor='val_loss', patience=100),
                 ModelCheckpoint(filepath='Biv_nonStationary_model.h5', monitor='val_loss', save_best_only=True)]
    result = model.fit(encoder_train, y_train, callbacks=callbacks, 
                       validation_data=(encoder_test,y_test), epochs = 200, batch_size = 64, verbose = 2)
    model = keras.models.load_model('Biv_nonStationary_model.h5')
    y_pred = model.predict(encoder_test)

    # Mean Squared Error
    mse_var1.append(mse(y_pred[:,0], y_test[:,0]))
    mse_var2.append(mse(y_pred[:,1], y_test[:,1]))

end_time = time.time()
print("%s seconds", end_time - start_time)

In [10]:
print("mean var 1 : ",np.mean(mse_var1))
print("mean var 2 : ",np.mean(mse_var2))
print("variance var 1 : ",np.var(mse_var1))
print("variance var 2 : ",np.var(mse_var2))


mean var 1 :  nan
mean var 2 :  nan
variance var 1 :  nan


In [11]:
# mse_var1



In [10]:
df_mse = pd.DataFrame(np.vstack((mse_var1,mse_var2)).T, columns = ["mse_var1","mse_var2"])

df_mse.to_csv("../plot_results/DeepKriging_nonstationary_mse.csv")

In [256]:
df = pd.read_csv("../test_data_nonstationary.csv")

In [258]:
s = np.vstack((df["x"],df["y"])).T
N = 400
num_basis = [2**2,5**2,9**2]
# num_basis = [2**2,5**2,9**2]
knots_1d = [np.linspace(0,1,int(np.sqrt(i))) for i in num_basis]
#knots_1d = [np.linspace(0,1,i) for i in num_basis]
##Wendland kernel
K = 0
phi = np.zeros((N, sum(num_basis)))
for res in range(len(num_basis)):
    theta = 1/np.sqrt(num_basis[res])*2.5
    knots_s1, knots_s2 = np.meshgrid(knots_1d[res],knots_1d[res])
    knots = np.column_stack((knots_s1.flatten(),knots_s2.flatten()))
    for i in range(num_basis[res]):
        d = np.linalg.norm(s-knots[i,:],axis=1)/theta
        for j in range(len(d)):
            if d[j] >= 0 and d[j] <= 1:
                phi[j,i + K] = (1-d[j])**6 * (35 * d[j]**2 + 18 * d[j] + 3)/3
            else:
                phi[j,i + K] = 0
    K = K + num_basis[res]

In [260]:
y_val = np.array(df[["var1","var2"]])

In [259]:
y_pred = model.predict(phi)

In [261]:
print(mse(y_pred[:,0], y_val[:,0]))
print(mse(y_pred[:,1], y_val[:,1]))

0.0004940607382401042
0.0009199529709178571


In [262]:
df_pred = pd.DataFrame(y_pred, columns = ["var1","var2"])

In [263]:

df_pred.to_csv("../plot_results/nonstationary_predictions.csv")

mean var 1 :  0.22820623264308693
mean var 2 :  0.48655204661394913
variance var 1 :  0.0366674693298505
variance var 2 :  0.18084126005806087

In [32]:
df_train_loc = pd.DataFrame(s_train, columns = ["x","y"])
df_train_val = pd.DataFrame(y_train, columns = ["var1","var2"])
df_train = pd.concat([df_train_loc,df_train_val], axis = 1)
df_train.to_csv("2d_biv_nonStationary_training.csv")

df_test_loc = pd.DataFrame(s_test, columns = ["x","y"])
df_test_val = pd.DataFrame(y_test, columns = ["var1","var2"])
df_test = pd.concat([df_test_loc,df_test_val], axis = 1)
df_test.to_csv("2d_biv_nonStationary_testing.csv")

In [3]:
df_loc = pd.DataFrame(s,columns = ["x","y"])
df_val = pd.DataFrame(y, columns = ["var1","var2"])
df = pd.concat([df_loc,df_val], axis = 1)
df.to_csv("../synthetic_data_simulations/2d_biv_nonStationary_1200.csv")

In [240]:
df.head()

,x,y,var1,var2
0,0.000000,0.0,-1.201086,-0.204747
1,0.025641,0.0,-1.382359,-0.501741
2,0.051282,0.0,-0.867809,0.188713
3,0.076923,0.0,-0.103645,1.105079
4,0.102564,0.0,0.395502,1.406360
